<a href="https://colab.research.google.com/github/markusMiksa1304/pubColab/blob/main/Kopie_von_MarkusErweiterung_BQ_Workshop_Telefonica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Intro

**Colab Authentication**

Run this first if you want to use the interactive Queries (not needed for read only!)

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
%%bigquery --project tefgfs
#https://towardsdatascience.com/leveraging-bigquery-with-google-analytics-data-b6085d8415b


WITH
  dateRangeData AS (
  SELECT
    *
  FROM
    `tefgfs.103498486.ga_sessions_*`
  WHERE
    _TABLE_SUFFIX BETWEEN "20200528" AND "20200628"
    )


Select
Count ( Distinct(
CASE
 WHEN totals.visits=1 THEN   #warum 
      CONCAT( fullvisitorid,"-",CAST(visitNumber AS string),"-",CAST(visitStartTime AS string))    
else ""    
End )) as Sessions
from
dateRangeData#, UNNEST(hits) AS hits#,

In [ ]:
%%bigquery --project tefgfs
#https://www.bounteous.com/insights/2016/06/23/number-sessions-day/

WITH
  dateRangeData AS (
  SELECT
    *
  FROM
    `tefgfs.103498486.ga_sessions_*`
  WHERE
    _TABLE_SUFFIX BETWEEN "20200528" AND "20200628"
    )
    
    SELECT  
       SUM(totals.visits) AS sessions
FROM dateRangeData
#GROUP BY date

In [ ]:
%%bigquery --project tefgfs
#https://www.bounteous.com/insights/2016/06/23/number-sessions-day/

WITH
  dateRangeData AS (
  SELECT
    *
  FROM
    `tefgfs.103498486.ga_sessions_*`
  WHERE
    _TABLE_SUFFIX BETWEEN "20200528" AND "20200628"
    )
    Select
  Count ( Distinct(
CASE
 WHEN totals.visits=1 THEN   #warum 
      CONCAT( fullvisitorid,"-",CAST(visitNumber AS string),"-",CAST(visitStartTime AS string))    
else ""    
End )) as Sessions
from
dateRangeData#, UNNEST(hits) AS hits#,

In [ ]:
%%bigquery --project tefgfs
#https://towardsdatascience.com/leveraging-bigquery-with-google-analytics-data-b6085d8415b


WITH
  dateRangeData AS (
  SELECT
    *
  FROM
    `tefgfs.103498486.ga_sessions_*`
  WHERE
    _TABLE_SUFFIX BETWEEN "20200528" AND "20200628"
    )
SELECT
  
  CONCAT( fullvisitorid,"-",CAST(visitNumber AS string),"-",CAST(visitStartTime AS string),"-",h.page.pagePath)    AS sessionId,
  date,
  h.hitNumber,
  
  h.page.pagePath,
  h.type,
  h.time,
  t.totals.visits,
  t.totals.hits,
  t.totals.pageviews
FROM
 dateRangeData AS t,
  t.hits AS h
  where h.type = "PAGE" and h.hitNumber=1
ORDER BY
  1,
  2 ASC
LIMIT 100

google drive mounten  .. some colab tips   https://towardsdatascience.com/getting-started-with-google-colab-f2fff97f594c


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!ls "/content/gdrive/My Drive/"

In [ ]:
!wget -cq https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip
!unzip -qq flower_data.zip
#That will give you Udacity’s flower data set in seconds

**Helpful resources:**
* WEB-UI: https://console.cloud.google.com/bigquery
* Query syntax: https://cloud.google.com/bigquery/docs/reference/standard-sql/query-syntax
* Functions and operators: https://cloud.google.com/bigquery/docs/reference/standard-sql/functions-and-operators
* GA BQ Schema: https://support.google.com/analytics/answer/3437719?hl=en
* BigQuery GA Cookbook:
	https://support.google.com/analytics/answer/4419694?hl=en
* BigQuery Pricing:
	https://cloud.google.com/bigquery/#pricing


---



**Exercises**

BQ Exercise 1: https://docs.google.com/document/d/15G3-cuk06asdkgz8PXvnFqOCFMPQaofZRugL3t_Cm0g/edit?usp=sharing




BQ Exercise 2: https://docs.google.com/document/d/11EuTkc4YNzZoAMAJ20IYh41zYPGaoZVadgocSLLk_FU/edit?usp=sharing

---



#**First Example Queries**

**Example 1:**  DO NOT USE Select * 

It can easily create huge queries!

In [ ]:
%%bigquery --project tefgfs df
SELECT * FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20170801`



---


**Example 2:**

Instead select the fields you need in before!
Like this sessions metric (for session scope!)

In [ ]:
%%bigquery --project tefgfs 
SELECT sum(totals.visits) as sessions FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20170801`

In [ ]:
%%bigquery --project tefgfs 

WITH
  dateRangeData AS (
  SELECT
    *
  FROM
    `tefgfs.103498486.ga_sessions_*`
  WHERE
    _TABLE_SUFFIX BETWEEN "20200528" AND "20200603"
    )
SELECT sum(totals.visits) as sessions from dateRangeData    

In [ ]:
%%bigquery --project tefgfs 

WITH date_range as 
(
SELECT * FROM  `tefgfs.103498486.ga_sessions_*`
WHERE _TABLE_SUFFIX = FORMAT_DATE("%Y%m%d", DATE_SUB('2020-09-05', INTERVAL 3 DAY))  #CURRENT_DATE() 
)
SELECT sum(totals.visits) as sessions from date_range  



---


**Example 3:**

Add further fields for dimensions

In [ ]:
%%bigquery --project tefgfs df
SELECT
  trafficSource.source,
  sum(totals.visits) as sessions
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20170801`
GROUP BY 1
LIMIT 10

Add ORDER BY to get the values in the format you might be used to from GA:

In [ ]:
%%bigquery --project tefgfs df
SELECT
  trafficSource.source,
  sum(totals.visits) as sessions
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20170801`
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10

Add further dimensions:

In [ ]:
%%bigquery --project tefgfs df
SELECT
  trafficSource.source,
  trafficSource.medium,
  sum(totals.visits) as sessions
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20170801`
GROUP BY 1,2
ORDER BY 3 DESC
LIMIT 10



---


**Example 4:**

Unnest repeated fields like hits:

In [ ]:
%%bigquery --project tefgfs df

SELECT
  h.page.pagePath,
  COUNT(*) AS pageviews
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20170801` AS t,
  t.hits AS h
WHERE
  h.type = 'PAGE'
GROUP BY
  1
ORDER BY
  2 DESC
LIMIT
 10



---


**Example 5:**

But be carefull when mixing hit scoped dimensions with metrics. The totals fields are false friends here, as they can only be used on session scope!
We need to recalculate the metrics on other scopes:

In [ ]:
%%bigquery --project tefgfs df

SELECT
  date,
  pagePath,
  SUM(visits) AS wrongSessionsMetric,
  COUNT(DISTINCT sessionid) sessions,
  SUM(pageviews) AS wrongPageviewsMetric,
  COUNT(DISTINCT CONCAT(sessionid,CAST(hitnumber AS STRING))) AS pageViews
FROM (
  SELECT
    date,
    CONCAT(t.fullVisitorId, CAST(t.visitStartTime AS STRING)) AS sessionId,
    h.hitNumber,
    h.page.pagePath,
    h.type,
    t.totals.visits,
    t.totals.hits,
    t.totals.pageviews
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20170801` AS t,
    t.hits AS h )
GROUP BY
  1,
  2
ORDER BY
  sessions DESC,
  pageViews DESC
LIMIT
 10



---


**Example 6:**

A sessionId is helpful to identify sessions:

In [ ]:
%%bigquery --project tefgfs 

SELECT
  CONCAT(t.fullVisitorId, CAST(t.visitStartTime AS STRING)) AS sessionId,
  h.hitNumber,
  h.page.pagePath,
  h.type,
  t.totals.visits,
  t.totals.hits,
  t.totals.pageviews
FROM
 `bigquery-public-data.google_analytics_sample.ga_sessions_20170801` AS t,
  t.hits AS h
ORDER BY
  1,
  2 ASC
LIMIT 10



---

**Example 7: Relative Date Range**

Instead of having to define fixed dates, you can also create date range relative to the current date.
Helpful, e.g. for scheduled repeated queries.

In [ ]:
%%bigquery --project tefgfs df

SELECT
  date,
  SUM(totals.visits) AS visits,
  SUM(totals.pageviews) AS pageviews,
  SUM(totals.transactions) AS transactions,
  SUM(totals.transactionRevenue)/1000000 AS revenue
FROM
  `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
  _TABLE_SUFFIX BETWEEN FORMAT_DATE('%Y%m%d',DATE_SUB(CURRENT_DATE(), INTERVAL 1095 DAY))
  AND FORMAT_DATE('%Y%m%d',DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY))
GROUP BY
  date
ORDER BY
  date ASC
  
LIMIT 10



---


**Example 8: Hit Scoped Custom Dimension:**

In [ ]:
%%bigquery --project tefgfs df

SELECT
  (SELECT value from h.customDimensions WHERE index=5) as cd5Hit
FROM
  `bigquery-public-data.google_analytics_sample.ga_sessions_20170801` as t, t.hits h
LIMIT
  10



---


**Example 9: Session/User Scoped Custom Dimension:**

In [ ]:
%%bigquery --project tefgfs df

SELECT
  (SELECT value from t.customDimensions WHERE index=4) as cd4
FROM
   `bigquery-public-data.google_analytics_sample.ga_sessions_20170801` as t
LIMIT
  10



---

**Example 10: JOINS**

In [ ]:
%%bigquery --project tefgfs df
SELECT
  year,
  mo,
  da,
  stn,
  name,
  country,
  temp,
  fog,
  rain_drizzle,
  snow_ice_pellets,
  hail,
  thunder
FROM
  `bigquery-public-data.noaa_gsod.gsod*` AS gsod
JOIN
  `bigquery-public-data.noaa_gsod.stations` AS stations
ON
  gsod.stn = stations.usaf
WHERE
  _TABLE_SUFFIX BETWEEN "2015"
  AND "2019"
ORDER BY
  year,
  mo,
  da
LIMIT
  10



---

**Example 11: GA BounceRate per Device**

In [ ]:
%%bigquery --project tefgfs df

SELECT
  deviceCategory,
  bounces,
  sessions,
  (bounces / sessions)*100 bounceRate
FROM (
  SELECT
    device.deviceCategory AS deviceCategory,
    SUM(totals.bounces) AS bounces,
    SUM(totals.visits) AS sessions
  FROM
    `bigquery-public-data.google_analytics_sample.ga_sessions_20170801`
  GROUP BY
    1 )
LIMIT 
10



---

**Example 12: GA BounceRate per Source**

In [ ]:
%%bigquery --project tefgfs df

SELECT
  source,
  total_visits,
  total_no_of_bounces,
  ( ( total_no_of_bounces / total_visits ) * 100 ) AS bounce_rate
FROM (
  SELECT
    trafficSource.source AS source,
    COUNT ( trafficSource.source ) AS total_visits,
    SUM ( totals.bounces ) AS total_no_of_bounces
  FROM
    `bigquery-public-data.google_analytics_sample.ga_sessions_*`
  WHERE
    _TABLE_SUFFIX BETWEEN '20170701'
    AND '20170731'
  GROUP BY
    source )
ORDER BY
  total_visits DESC
LIMIT 
10



---

**Example 13: GA Page Funnel**

In [ ]:
%%bigquery --project tefgfs df

SELECT
  pagePath,
  next_pagepath,
  next_next_pagepath,
  COUNT(*) as combinations
FROM (
  SELECT
    pagePath,
    LEAD(pagePath) OVER (PARTITION BY fullVisitorId, visitStartTime ORDER BY hitNumber) AS next_pagepath,
    LEAD(pagePath,2) OVER (PARTITION BY fullVisitorId, visitStartTime ORDER BY hitNumber) AS next_next_pagepath
  FROM (
    SELECT
      fullVisitorId,
      visitStartTime,
      hits.page.pagePath,
      hits.hitNumber
    FROM
      `bigquery-public-data.google_analytics_sample.ga_sessions_*` AS GA,
      UNNEST(GA.hits) AS hits
    WHERE
      hits.type = 'PAGE'
      AND _TABLE_SUFFIX BETWEEN "20170701"
      AND "20170801" ) )
GROUP BY
  1,
  2,
  3
ORDER BY
  4 DESC
LIMIT 
10



---

**Example 14: GA Next Pageview**

In [ ]:
%%bigquery --project tefgfs df

SELECT
  fullVisitorId,
  visitStartTime,
  pagePath,
  hit_time,
  LEAD(pagePath) OVER (PARTITION BY fullVisitorId, visitStartTime ORDER BY hit_time) AS next_pagepath,
  LEAD(hit_time) OVER (PARTITION BY fullVisitorId, visitStartTime ORDER BY hit_time) AS next_pageview_hit_time,
  LEAD(pagePath,2) OVER (PARTITION BY fullVisitorId, visitStartTime ORDER BY hit_time) AS next_next_pagepath,
  LEAD(hit_time,2) OVER (PARTITION BY fullVisitorId, visitStartTime ORDER BY hit_time) AS next_next_pageview_hit_time
FROM (
  SELECT
    fullVisitorId,
    visitStartTime,
    hits.page.pagePath,
    hits.time / 1000 AS hit_time
  FROM
    `bigquery-public-data.google_analytics_sample.ga_sessions_20160801` AS GA,
    UNNEST(GA.hits) AS hits
  WHERE
    hits.type = 'PAGE')
 LIMIT 10



---

**Example 15: GA Next Event**

In [ ]:
%%bigquery --project tefgfs df

SELECT
  eventAction,
  eventLabel,
  nextEventAction,
  nextEventLabel,
  COUNT(*) AS count
FROM (
  SELECT
    fullVisitorId,
    visitId,
    hitNumber,
    eventInfo.eventCategory,
    eventInfo.eventAction,
    eventInfo.eventLabel,
    LEAD(eventInfo.eventAction,1) OVER (PARTITION BY fullVisitorId, visitId ORDER BY hitNumber) AS nextEventAction,
    LEAD(eventInfo.eventLabel,1) OVER (PARTITION BY fullVisitorId, visitId ORDER BY hitNumber) AS nextEventLabel
  FROM
    `bigquery-public-data.google_analytics_sample.ga_sessions_20170801`,
    UNNEST(hits) AS hit
  WHERE
    hit.type = 'EVENT')
GROUP BY
  1,
  2,
  3,
  4
ORDER BY
  5 DESC
LIMIT
10



---

**Example 16: Landing Page**

In [ ]:
%%bigquery --project tefgfs 
WITH
  dateRangeData AS (
  SELECT
    *
  FROM
    `tefgfs.103498486.ga_sessions_*` 
  WHERE
    _TABLE_SUFFIX BETWEEN "20200528" AND "20200529"
    limit 2000
    )

SELECT DISTINCT 
  landingpage,
  COUNT(DISTINCT(sessionId)) sessions,
  SUM(bounces) bounces,
  AVG(pageviews) avg_pages,
  (SUM(tos)/COUNT(DISTINCT(sessionId)))/60 session_duration,
  isSessionCommProduct,
  SessionCommProduct
FROM 
(
  SELECT DISTINCT 
    CONCAT(fullVisitorId,CAST(visitStartTime AS STRING)) sessionId,
    (SELECT page.pagePath FROM UNNEST(hits) WHERE hitnumber = ( SELECT MIN(hitnumber) FROM UNNEST(hits) WHERE type = 'PAGE')) landingpage,
    totals.bounces bounces,
    totals.timeonsite tos,
    totals.pageviews pageviews,
 
  (SELECT
        MAX(IF(REGEXP_CONTAINS(v2ProductCategory, r"postpaid|hw-only|smartphone") and not REGEXP_CONTAINS(v2ProductCategory, r"-vvl|-tw|pack") , v2ProductCategory, "")) 
          FROM UNNEST(hits) as h, UNNEST(product) as p) as SessionCommProduct, --IF 1=1 for null value handling


    IF((SELECT
        MAX(IF(REGEXP_CONTAINS(v2ProductCategory, r"postpaid|hw-only|smartphone") and not REGEXP_CONTAINS(v2ProductCategory, r"-vvl|-tw|pack") , 1, 0)) as isSessionCommProduct
          FROM UNNEST(hits) as h, UNNEST(product) as p) = 1, 1, 0) as isSessionCommProduct, --IF 1=1 for null value handling

   FROM
 # `bigquery-public-data.google_analytics_sample.ga_sessions_20170801`
 dateRangeData, unnest(hits) as h
WHERE
    totals.visits = 1 
  )  #es gibt überhaupt interaction events)
  WHERE  isSessionCommProduct =1 
GROUP BY
  1,isSessionCommProduct,SessionCommProduct
ORDER BY
  2 DESC
LIMIT 
100



---

**Example 17: Channel Grouping via External JS File UDF**

As discussed, this is not the perfect way to do the grouping, because of performance lagging with UDF.
Also the CG is far from perfect in its definition and rather for demo here.

**JS file channelgrouping.js**

```
function getCG(source, medium, adNetworkType) {
  try {
    if ((source == '(direct)' && medium == '(not set)') || medium == '(none)') {
      return 'Direct';
    } else if (medium == 'organic') {
      return 'Organic Search';
    } else if (/^(social|social-network|social-media|sm|social network|social media)$/.test(medium) ||
              medium == 'referral' && 
              /(.*facebook.*|.*xing.*|.*plus\.(url\.)?google.*|.*linkedin.*|.*twitter.*|^t.co$|^lnkd.in$)/.test(source)) {
      return 'Social';
    } else if (medium == 'email') {
      return 'Email';
    } else if (medium == 'affiliate') {
      return 'Affiliates';
    } else if (medium == 'referral') {
      return 'Referral';
    } else if (/^(cpc|ppc|paidsearch)$/.test(medium) && adNetworkType != 'Content') {
      return 'Paid Search';
    } else if (/^(cpv|cpa|cpp)$/.test(medium)) {
      return 'Other Advertising';
    } else if (/^(display|cpm|banner)$/.test(medium) || adNetworkType == 'Content') {
      return 'Display';
    } else {
      return '(Other)';
    }
  } catch (ex) {
    return 'error';
  }
}
```



You might not be able to run this code directly, without storing the JS file on GCS in your project first and linking to it.

In [ ]:
%%bigquery --project tefgfs df


CREATE TEMP FUNCTION
  customCG(source STRING,
    medium STRING,
    adNetworkType STRING)
  RETURNS STRING
  LANGUAGE js AS """
    return getCG(source, medium, adNetworkType);
"""
OPTIONS
  ( library="gs://go-de-bq-demo/udf/channelgrouping.js" );
  
SELECT
  fullvisitorid,
  visitid,
  trafficSource.source,
  trafficSource.medium,
  trafficSource.adwordsClickInfo.adNetworkType,
  trafficSource.isTrueDirect,
  channelGrouping,
  customCG(trafficSource.source,
    trafficSource.medium,
    trafficSource.adwordsClickInfo.adNetworkType) customChannelGrouping
FROM
  `bigquery-public-data.google_analytics_sample.ga_sessions_20170801`
LIMIT
10



---


# **Exercise Solutions**

**Solution Exercise 2:**
  
Query 1:

In [ ]:
%%bigquery --project tefgfs 
SELECT
  date,
  COUNT(DISTINCT fullVisitorId) as users,
  SUM(totals.visits) AS sessions,
  SUM(totals.hits) AS hits,
  SUM(totals.pageviews) AS pageviews,
  SUM(totals.transactions) AS transactions,
  SUM(totals.totalTransactionRevenue)/1000000 AS revenue
FROM
  `tefgfs.103498486.ga_sessions_*`
WHERE
  _TABLE_SUFFIX BETWEEN "20190528" AND "20190603"
GROUP BY
  date
ORDER BY
  date



---


**Solution Exercise 2:**

Query 2:

In [ ]:
%%bigquery --project tefgfs df
WITH
  date_range AS (
  SELECT
    *
  FROM
    `tefgfs.103498486.ga_sessions_*`
  WHERE
    _TABLE_SUFFIX BETWEEN "20190528" AND "20190603")
    
SELECT
  q1.date,
  q1.sessions,
  q1.hits,
  q1.pageviews+q2.events AS pageviewsPlusEvents,
  q1.pageviews,
  q2.events,
  q1.transactions,
  q1.revenue
FROM (
  SELECT
    date,
    SUM(totals.visits) AS sessions,
    SUM(totals.hits) AS hits,
    SUM(totals.pageviews) AS pageviews,
    SUM(totals.transactions) AS transactions,
    SUM(totals.totalTransactionRevenue)/1000000 AS revenue
  FROM
    date_range
  GROUP BY
    date ) AS q1
JOIN (
  SELECT
    date,
    COUNT(*) AS events
  FROM
    date_range AS t,
    t.hits AS h
  WHERE
    h.type = "EVENT"
  GROUP BY
    t.date ) AS q2
ON
  q1.date = q2.date
ORDER BY
  1

**Send email**

In [ ]:
#das funktioniert !!!
#https://realpython.com/python-send-email/
#
#https://developers.google.com/gmail/api/auth/web-server?hl=en_GB


import smtplib, ssl

port = 465  # For SSL
smtp_server = 
sender_email =   # Enter your address
   #dafür hab ich in meinem gmail security settings extra dieses app-passwort für mail  creiert 
message = """\
Subject: Hi there

This message2 is sent from Python."""

context = ssl.create_default_context()
with smtplib.SMTP_SSL(smtp_server, port, context=context) as server:
    server.login(sender_email, password)
    server.sendmail(sender_email, receiver_email, message)

In [ ]:
%%bigquery --project tefgfs 
SELECT 
 IF(totals.transactions IS NULL, 0, 1) AS label,
  IFNULL(device.operatingSystem, "") AS os,
  device.isMobile AS is_mobile,
  IFNULL(geoNetwork.country, "") AS country,
  IFNULL(totals.pageviews, 0) AS pageviews
FROM
    `tefgfs.103498486.ga_sessions_*`
  WHERE
    _TABLE_SUFFIX BETWEEN "20190528" AND "20190529"

,label,os,is_mobile,country,pageviews
0,0,Android,True,Germany,0
1,0,Android,True,Germany,0
2,0,Android,True,Germany,0
3,0,Windows,False,Germany,0
4,0,Windows,True,Germany,0
...,...,...,...,...,...
868847,0,Android,True,Germany,1
868848,0,iOS,True,Germany,2
868849,0,Android,True,Germany,3
868850,0,Android,True,Germany,2


In [ ]:
%%bigquery --project tefgfs 
select * from  `tefgfs.ReportingData.mopGaSourcev6_o2_20200929` where gaTransactId is not NULL Limit 100

,sessionDate,rowId,visitId,GCLID,gaTransactId,transactionItemCats,transactionItemVariants,productCheckoutPromocode,quantity,sessionDeviceCategory,sessionDurationMS,sessionTotalPageviewCount,sessionTotalInteractionCount,sessionTotalProductViewCount,sessionTotalAddToBasketCount,channelGrouping,utmContent,utmCampaign,isTrueDirectSession,utmKeyword,utmMedium,utmSource,trboSegment_strongestAffinityManufacturer_User,trboSegment_strongestAffinityClientType_User,trboSegment_strongestAffinityGeneralTariff_User,trboSegment_strongestAffinityOfferType_User,trboSegment_latestAffinityManufacturer_User,trboSegment_latestAffinityClientType_User,trboSegment_latestAffinityGeneralTariff_User,trboSegment_latestAffinityOfferType_User,highInterest_hardwareName,highInterest_hardwareName_score,highInterest_productName,highInterest_productName_score,gaChurnLevel,isSessionCommProduct,isSessionCd70Postpaid,landingpage,isBounce,isServiceTraffic,isDSLSession,isPixieSession,isPixieOnlySession,isDirectBuySession,isVVLSession,isCustomerSession,isCartAdd,isEShopTransaction,ProductCategoryFilter_PO,LandingpageFilter_PO,SourceFilter_PO,ChannelgroupingFilter_PO
0,20200929,3669ebb6-8662-4b0c-b3b4-e6acdafc61e2,1601361683,None,bk-econtract-o2online-1601362045.1613635767-1,postpaid_sim-only-vvl,vvl,None,1,desktop,416538,34,132,3,0,Direct,(not set),(not set),True,(not set),(none),(direct),None,None,None,None,None,None,None,None,tariff-only,108,o2 Free S (2019),62,0,1,1,/,0,1,0,0,0,0,1,1,0,0,0,0,0,0
1,20200929,13905578-c0cb-41f7-8288-6183cc2d84d1,1601366320,None,bk-econtract-o2online-1601366812.1664783989-1,postpaid_sim-only-vvl,vvl,None,1,desktop,504056,16,83,3,0,Other_Referral,(not set),(not set),None,(not set),referral,trendupdeal.com,None,None,None,None,None,None,None,None,tariff-only,126,o2 Free Unlimited Basic,74,0,1,1,/,0,1,0,0,0,0,1,1,0,0,0,0,0,0
2,20200929,2eab297f-05a4-4ac3-afdf-e94b835709b5,1601401338,None,bk-econtract-o2online-1601402474.328484994-1,postpaid_sim-only-vvl,vvl,None,1,desktop,1627089,18,103,4,0,Direct,(not set),(not set),True,(not set),(none),(direct),None,None,None,None,None,None,None,None,tariff-only,92,o2 Free M (2018),62,0,1,1,/,0,1,0,0,0,0,1,1,0,0,0,0,0,0
3,20200929,13932dd0-b65f-43fe-be85-a2c44237588f,1601382529,None,bk-econtract-o2online-1601384040.1959566254-1,postpaid_sim-only-vvl,vvl,None,1,desktop,1859954,46,166,5,0,SEO,(not set),(not set),None,(not provided),organic,google,None,None,None,None,None,None,None,None,tariff-only,130,o2 Free S (2018),64,0,1,1,/,0,1,0,0,0,0,1,1,0,0,0,0,0,0
4,20200929,7417ef0e-e853-4204-a7fa-a0b977919833,1601364527,None,o2-B-pAuqk5a0TjaibXMweHDsaA-1,postpaid_sim-only,(not set),NO_PROMOTION,1,desktop,164412,7,52,0,0,Direct,(not set),(not set),True,(not set),(none),(direct),None,None,None,None,None,None,None,None,tariff-only,120,bestandskunden-o2-free-starter-flex-online-par...,120,0,1,1,/,0,0,0,0,0,0,0,1,1,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,20200929,8fafbce8-7349-436e-9a0b-82e29b5ae8dd,1601401911,None,o2-N-T2-ifUztToKRSzYslruriw-1,hw-only_smartphone,(not set),no_campaign,1,desktop,732703,25,76,16,3,SEO,(not set),(not set),None,(not provided),organic,bing,None,None,None,None,None,None,None,None,pixie_privatkunden-apple-iphone-11-pro-max-64g...,450,hw-only,330,0,1,1,/pixie/,0,0,0,1,1,0,0,0,0,0,0,0,0,0
96,20200929,3210315b-abde-4ec6-b263-6907450fe701,1601386739,None,o2-N-0B9uUFJYRnG-WhHDOYzYmg-1,hw-only_smartphone,(not set),no_campaign,1,mobile,358153,20,46,5,2,SEO,(not set),(not set),True,(not provided),organic,google,None,None,None,None,None,None,None,None,pixie_privatkunden-apple-iphone-11-64gb-schwar...,270,hw-only,290,0,1,1,/pixie/,0,0,0,1,1,0,0,0,0,0,0,0,0,0
97,20200929,576399d9-e14d-433e-88a3-1e36b5ae873f,1601412666,Cj0KCQjwtsv7BRCmARIsANu-CQcvDk7Xu7i-hXnj262Cgu...,o2-C-zhpvODqUSs6P0y1HUWMWWg-1,postpaid_sim-only,(no

**Geralds SQL**

In [ ]:
%%bigquery --project tefgfs 
WITH 
date_range as(
              Select * FROM `tefgfs.69128167.ga_sessions_*`
              WHERE _TABLE_SUFFIX = "20200920" #FORMAT_DATE("%Y%m%d", DATE_SUB(@run_date, INTERVAL 1 DAY))
),
-----------------
-- SESSION FIELDS
-----------------
session_view as(
              SELECT
                  date,
                  visitid,
                  clientid,
                  fullvisitorid,      
                  visitstarttime,
                  trafficSource.adwordsClickInfo.gclid AS GCLID,
                  device.deviceCategory AS sessionDeviceCategory,
                  trafficSource.adContent AS utmContent,
                  trafficSource.campaign AS utmCampaign,
                  trafficSource.isTrueDirect AS isTrueDirectSession,
                  trafficSource.keyword AS utmKeyword,
                  trafficSource.medium AS utmMedium,
                  trafficSource.source AS utmSource,
                  (SELECT STRING_AGG(DISTINCT cd.value) FROM UNNEST (hits) AS h, UNNEST(h.customdimensions) AS cd WHERE cd.index = 138) AS Customer_Group_URL_Hit,
                  (SELECT STRING_AGG(DISTINCT cd.value) FROM UNNEST(customdimensions) AS cd WHERE cd.index = 155) AS URLvoNumber_session,
                  (SELECT STRING_AGG(DISTINCT cd.value) FROM UNNEST (hits) AS h, UNNEST (h.customdimensions) AS cd WHERE cd.INDEX = 193) AS Application_Hit,
              FROM date_range
),
------------------------------------
-- SUBQUERY FOR PRODUCT DETAIL VIEWS
------------------------------------
detail_view as(
              SELECT
                date,
                visitid,
                clientID,
                fullvisitorid,
                visitstarttime,
                hp.v2ProductName AS Tarif_DetailView,
                count (hp.v2ProductName) as DetailView_cnt
              FROM date_range, UNNEST (hits) AS h, UNNEST (h.product) as hp
              WHERE h.eCommerceAction.action_type = "2" AND (hp.isImpression IS NULL)
              Group by 1,2,3,4,5,6
),
----------------------------
-- SUBQUERY FOR TRANSACTIONS
----------------------------
transaction_view as( -- SubQuery for Transaction Information
             SELECT
                date,
                visitid,
                clientID,
                fullvisitorid,
                visitstarttime,
                hp.v2ProductName AS Tarif_Transaction,
                h.transaction.transactionId AS gaTransactId,
                hp.productQuantity as Quantity,
            FROM date_range, UNNEST (hits) AS h, UNNEST (h.product) as hp
            WHERE h.eCommerceAction.action_type = "6" AND (hp.isImpression IS NULL)
),
----------------------------------------------------------------
-- COMBINATION OF SESSIONS WITH DETAIL VIEWS AND/OR TRANSACTIONS
----------------------------------------------------------------
detail_transaction as(
            SELECT
                a.*,
                Tarif_detailView,
                DetailView_cnt,
                "" AS Tarif_Transaction,
                "" AS gaTransactId,
                null as Quantity
            FROM session_view as a

          INNER JOIN(

            SELECT
                *
            FROM detail_view) as b ON a.date = b.date AND a.visitid = b.visitid AND a.clientid = b.clientid AND a.fullvisitorid = b.fullvisitorid AND a.visitstarttime = b.visitstarttime

          UNION ALL

            SELECT
                a.*,
                "" as Tarif_detailView,
                null as DetailView_cnt,
                Tarif_transaction,
                gaTransactId,
                Quantity
            FROM session_view as a
            
          INNER JOIN(
          
            SELECT
                *
            FROM transaction_view) as c ON a.date = c.date AND a.visitid = c.visitid AND a.clientid = c.clientid AND a.fullvisitorid = c.fullvisitorid AND a.visitstarttime = c.visitstarttime
),
----------------------------------------------
-- SESSIONS WITHOUT DETAIL VIEW OR TRANSACTION
----------------------------------------------
session_only as(
            SELECT
                s.*,
                "" as Tarif_detailView,
                null as DetailView_cnt,
                "" AS Tarif_Transaction,
                "" AS gaTransactId,
                null as Quantity
            FROM session_view as s

          LEFT JOIN
          
            (SELECT
            *
            FROM detail_transaction) as dt ON dt.date = s.date AND dt.visitid = s.visitid AND dt.clientId = s.clientId AND dt.fullvisitorid = s.fullvisitorid AND dt.visitstarttime = s.visitstarttime
            WHERE dt.date is null AND dt.visitid is null and dt.clientid is null and dt.fullvisitorid is null and dt.visitstarttime is null
)


-----------------------------------------------------------------------------------------------------------------
-- MAIN QUERY TO COMBINE SESSIONS WITH DETAILVIEWS, TRANSACTIONS AND REST OF SESSIONS WITHOUT PRODUCT INTERACTION
-----------------------------------------------------------------------------------------------------------------
SELECT
    *
FROM session_only

UNION ALL

SELECT
    *
FROM detail_transaction
ORDER BY visitid

In [ ]:
%%bigquery --project tefgfs 

SELECT count(TRANSACTION_ID) FROM `tefgfs.MetaLookupData.NET_Conversions_20200930` 

In [ ]:
%%bigquery --project tefgfs 

SELECT * FROM `tefgfs.MetaLookupData.NET_Conversions_*`  where TRANSACTION_ID like "%o2-N-mQfFOLaUS-CclMGsNT-9Pg-1"

In [ ]:
%%bigquery --project tefgfs 
Select gaTransactId,TRANSACTION_ID,ORDER_STATUS 
from (
SELECT * FROM  (Select gaTransactId,sessionDate,visitId   From `tefgfs.ReportingData.mopGaSourcev6_o2_20200829`  where gaTransactId is not NULL  and gaTransactId <> "None" Limit 10) mopGA
LEFT  JOIN  (SELECT * FROM `tefgfs.MetaLookupData.NET_Conversions_*`   )  Activations
ON mopGA.gaTransactId = Activations.TRANSACTION_ID ) mopGA6plus  

**Alle transactionIds (left) werden in der Aktivierungstabelle (rechts) gefunden**


In [ ]:
%%bigquery --project tefgfs 

SELECT count(visitID)  as cd  FROM  (Select *   From `tefgfs.ReportingData.mopGaSourcev6_o2_20200829`  where gaTransactId is not NULL  and gaTransactId <> "None" ) mopGA
LEFT  JOIN  (SELECT * FROM `tefgfs.MetaLookupData.NET_Conversions_*`   )  Activations
ON mopGA.gaTransactId = Activations.TRANSACTION_ID  

In [ ]:
%%bigquery --project tefgfs 
SELECT count(visitID)  as cd  FROM  (Select *   From `tefgfs.ReportingData.mopGaSourcev6_o2_20200829`  where gaTransactId is not NULL  and gaTransactId <> "None" )

**Hinzufügen des Orderaktivierungs-Status zur mopGA**

In [ ]:
%%bigquery --project tefgfs 

SELECT *   EXCEPT (TRANSACTION_ID,UPLOAD_DATE	) FROM  (Select *   From `tefgfs.ReportingData.mopGaSourcev6_o2_20200829`  where gaTransactId is not NULL  and gaTransactId <> "None" ) mopGA
LEFT  JOIN  (SELECT * FROM `tefgfs.MetaLookupData.NET_Conversions_*`   )  Activations
ON mopGA.gaTransactId = Activations.TRANSACTION_ID   Where ORDER_STATUS <> "None" order by ORDER_STATUS

**Ansatz zur customer journey** https://towardsdatascience.com/visualising-how-users-start-their-journey-on-your-website-with-ga-bigquery-and-r-**aa56fa39e386**

In [ ]:
%%bigquery --project tefgfs 
SELECT t.page_path,
        t.second_page_path,
        t.third_page_path,
        t.fourth_page_path,
        #concact the 4 pages together with a hyphen if a next page exists
        CONCAT(t.page_path,IF(t.second_page_path IS NULL,"","-"),
        IFNULL(t.second_page_path,""),IF(t.third_page_path IS NULL,"","-"),
        IFNULL(t.third_page_path,""),IF(t.fourth_page_path IS NULL,"","-"),
        IFNULL(t.fourth_page_path,"")) AS full_page_journey,
        #Count total sessions that went down that path after landing
        count(sessionId) AS total_sessions

FROM (

SELECT
     #create session ID 
     CONCAT(fullVisitorId,"-",CAST(visitStartTime as string)) AS sessionId,
     #Get hitNumber for session e.g 1 is the first hit in the session
     hits.hitNumber,
     #Select first Page 
     hits.page.pagePath AS page_path,
     #Select second page
     LEAD(hits.page.pagePath) OVER (PARTITION BY fullVisitorId, visitStartTime ORDER BY hits.hitNumber) AS second_page_path,
     #Select third page
     LEAD(hits.page.pagePath,2) OVER (PARTITION BY fullVisitorId, visitStartTime ORDER BY hits.hitNumber) AS third_page_path,
     #Select fourth page
     LEAD(hits.page.pagePath,3) OVER (PARTITION BY fullVisitorId, visitStartTime ORDER BY hits.hitNumber) AS fourth_page_path
   FROM
     #tables with data from 17t-23rd July 2017
     #[bigquery-public-data:google_analytics_sample.ga_sessions_20170717],
      `tefgfs.103498486.ga_sessions_20200803` AS GA,
       Unnest (GA.hits) as hits
    WHERE
     #Only count page views
     hits.type="PAGE"
     
     ) t
     #Only select fors which include data from the first hit
     WHERE t.hits.hitNumber=1
     #Group by page of hit and all subsequent pages
     GROUP BY t.page_path,
              t.second_page_path,
              t.third_page_path,
              t.fourth_page_path,
              full_page_journey
     ORDER BY total_sessions DESC
     

In [ ]:
pageviews

Highly active users firebase

In [ ]:
%%bigquery --project tefgfs 
SELECT 
( SUM(total_pagesviews_per_user) / COUNT(users) ) AS avg_pageviews_per_user
FROM
(
SELECT
fullVisitorId AS users,
SUM(totals.pageviews) AS total_pagesviews_per_user
FROM `tefgfs.103498486.ga_sessions_*`
WHERE
_TABLE_SUFFIX BETWEEN '20200701' AND '20200831'
AND
totals.transactions >=1
GROUP BY
users )


In [ ]:
 %%bigquery --project tefgfs 
 
SELECT
    fullVisitorId AS id,
    visitId AS visitid,
    visitNumber AS visitnumber,
    h.hitNumber AS hitNumber,
   channelGrouping AS journey
  FROM
   (Select * From `tefgfs.103498486.ga_sessions_20200803`limit 200 ) AS GA,
       Unnest (GA.hits) as h
  WHERE
    h.type="PAGE"
    OR h.type="EVENT"
   

In [ ]:
%%bigquery --project tefgfs 
#standardSQL
WITH mm as (SELECT
    date
    fullVisitorId ,
    visitId AS visitid,
    visitNumber AS visitnumber,
    h.hitNumber AS hitNumber,
    channelGrouping AS journey
  FROM
   (Select * From `tefgfs.103498486.ga_sessions_20200803`limit 2000 ) AS GA,
       Unnest (GA.hits) as h
  WHERE
    h.type="PAGE"
    OR h.type="EVENT"
)

SELECT 
  fullVisitorId, visitId, 
  STRING_AGG(journey, ' -> ' ORDER BY visitNumber, hitnumber) journey_path
FROM (
  SELECT 
    fullVisitorId, visitId, 
    MIN(visitNumber) visitNumber, MIN(hitnumber) hitnumber, journey
  FROM (
    SELECT *, COUNTIF(journey = 'Lead') OVER(win) grp
    FROM mm
    WINDOW win AS (
      PARTITION BY fullVisitorId, visitId 
      ORDER BY visitNumber, hitnumber 
      ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
    )
  )
  WHERE grp = 0
  GROUP BY fullVisitorId, visitId, journey
)
GROUP BY fullVisitorId, visitId
   

In [ ]:
%%bigquery --project tefgfs 

SELECT
fullVisitorId AS visitId,
SUM(totals.pageviews) AS total_pagesviews_per_user
FROM `tefgfs.103498486.ga_sessions_*`
WHERE
_TABLE_SUFFIX BETWEEN '20200701' AND '20200702'
AND
totals.transactions >=1
GROUP BY
visitId 



*   Listeneintrag
*   customer journey
https://lynuhs.com/detailed-attribution-in-google-analytics-360/


In [ ]:
%%bigquery --project tefgfs 
WITH ga as 
(
SELECT * FROM  `tefgfs.103498486.ga_sessions_*`
WHERE _TABLE_SUFFIX = FORMAT_DATE("%Y%m%d", DATE_SUB('2020-09-05', INTERVAL 3 DAY))  #CURRENT_DATE() 
)

 SELECT
    CONCAT('id_', CAST(fullvisitorId AS STRING)) AS fullVisitorId,
    MAX(transactionId) AS transactionId,
    date,
    visitStartTime,
    channel
    FROM (
    SELECT
        fullVisitorId,
        transactionId,
        date,
        visitStartTime,
        CASE
        WHEN sm LIKE '%google%cpc%' AND (adn != 'content' OR adn IS NULL) THEN 'Google - Paid'
        WHEN sm LIKE '%bing%cpc%' THEN 'Bing - Paid'
        WHEN sm LIKE 'google%organic' THEN 'Google - Organic'
        WHEN sm LIKE 'bing%organic' THEN 'Bing - Organic'
        ELSE 'Other'
        END AS channel,
        channelGrouping as channel
        CASE
        WHEN MAX(transactionId) OVER (PARTITION BY fullVisitorId) IS NOT NULL THEN 1
        ELSE 0
        END AS convertingCustomer
    FROM (
        SELECT
        fullVisitorId,
        date,
        visitStartTime,
        CONCAT(trafficSource.source, ' / ', trafficSource.medium) AS sm,
        trafficSource.adwordsClickInfo.adNetworkType AS adn,
        hit.transaction.transactionId AS transactionId
        FROM
        ga, # Write your own IDs
        UNNEST(ga.hits) hit
        GROUP BY
        fullVisitorId,
        date,
        trafficSource.source,
        trafficSource.medium,
        visitStartTime,
        trafficSource.adwordsClickInfo.adNetworkType,
        hit.transaction.transactionId ))
    WHERE
    convertingCustomer = 1
    GROUP BY
    fullvisitorId,
    date,
    visitStartTime,
    channel
    ORDER BY
    fullVisitorId,
    visitStartTime


In [ ]:

 #standardSQL
%%bigquery --project tefgfs
 #standardSQL
with mdata as (
SELECT
fullVisitorId,
visitId,
visitNumber,
hits.hitNumber AS hitNumber,
case
    when IF(REGEXP_CONTAINS(page.pagePath, r"eshop.*basket|e-shop/warenkorb|e-shop/checkout|e-shop/bestellung|e-store/cart"), 1, 0) =1  then hits.page.pagePath 
 else
    "None"
end AS journey
FROM
  `tefgfs.103498486.ga_sessions_*`,
   UNNEST(hits) as hits

WHERE
    _TABLE_SUFFIX = FORMAT_DATE("%Y%m%d", DATE_SUB('2020-09-05', INTERVAL 3 DAY))  #CURRENT_DATE()
   AND
hits.type="PAGE"
ORDER BY
fullVisitorId,
visitId,
visitNumber,
hitNumber
)
Select fullVisitorId , 
 count(journey) as LenJourney,
 STRING_AGG(journey, ' -> ' ORDER BY visitNumber, hitnumber) journey_path,
 
from  mdata 
 where journey <>"None"
group by fullVisitorId


In [ ]:

# -- SubQuery for Transaction Information
%%bigquery --project tefgfs

 SELECT
                date,
                visitid,
                clientID,
                fullvisitorid,
                visitstarttime,
                hp.v2ProductName AS Tarif_Transaction,
                h.transaction.transactionId AS gaTransactId,
                hp.productQuantity as Quantity,
            FROM  `tefgfs.103498486.ga_sessions_20200605`, UNNEST (hits) AS h, UNNEST (h.product) as hp
            WHERE h.eCommerceAction.action_type = "6" AND (hp.isImpression IS NULL)

In [ ]:
%%bigquery --project tefgfs
 #standardSQL

with mdata as (
SELECT
fullVisitorId,
visitId,
visitNumber,
hits.hitNumber AS hitNumber,
case
    when IF(REGEXP_CONTAINS(page.pagePath, r"eshop.*basket|e-shop/warenkorb|e-shop/checkout|e-shop/bestellung|e-store/cart"), 1, 0) =1  then "atWarenkorb" 
    when IF(REGEXP_CONTAINS(page.pagePath, r"eshop.*confirmation|/pixie/confirmation/|/e-shop/bestellung/") OR
        eventInfo.eventCategory = "Summary Paynow", 1, 0) = 1 then "Bestellung" 
    when ecommerceaction.action_type = "0" then "productImpression"
    when ecommerceaction.action_type = "2" then "productView"
    when ecommerceaction.action_type = "3" then "addToBasket"
    when ecommerceaction.action_type = "5" then "checkOut"
    when ecommerceaction.action_type = "6" then "Kauf"
   
    else
    "None"
end AS journey
FROM
  `tefgfs.103498486.ga_sessions_*`,
   UNNEST(hits) as hits

WHERE
    _TABLE_SUFFIX = FORMAT_DATE("%Y%m%d", DATE_SUB('2020-09-05', INTERVAL 3 DAY))  #CURRENT_DATE()
   AND
hits.type="PAGE"
ORDER BY
fullVisitorId,
visitId,
visitNumber,
hitNumber
)
SELECT 
  fullVisitorId, visitId, 
  STRING_AGG(journey, ' -> ' ORDER BY visitNumber, hitnumber) journey_path
FROM (
  SELECT 
    fullVisitorId, visitId, 
    MIN(visitNumber) visitNumber, MIN(hitnumber) hitnumber, journey
  FROM (
    SELECT *, COUNTIF(journey = 'addToBasket') OVER(win) grp
    FROM mdata
    WINDOW win AS (
      PARTITION BY fullVisitorId, visitId 
      ORDER BY visitNumber, hitnumber 
      ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
    )
  )
  WHERE grp = 0
  GROUP BY fullVisitorId, visitId, journey
)
GROUP BY fullVisitorId, visitId

mopGA6

In [ ]:
#--mop gaSource query version: 6
#--die Query die aus den Rohdaten (103498486) die ReportingData.mopGaSourcev4_o2Segments_yyyyMMdd generiert. (das gleiche gibt es 1:1 für blau)

%%bigquery --project tefgfs
#https://github.com/Telefonica-Germany/TEFBigQuery/blob/master/mop-gaSource-v6-bl.sql

WITH date_range as 
(
SELECT * FROM  `tefgfs.103498486.ga_sessions_20200801` Limit 200000
#WHERE _TABLE_SUFFIX = FORMAT_DATE("%Y%m%d", DATE_SUB(@run_date, INTERVAL 1 DAY)) 
)
,
 mopGA as
(
SELECT
  s.date,
  s.clientId,
  s.visitId,
  trafficSource.adwordsClickInfo.gclid AS GCLID, 
  tr.gaTransactId gaTransactId,
  transactionItemCats,
  productCheckoutPromocode,
  device.deviceCategory AS sessionDeviceCategory,
  channelGrouping,
  trafficSource.adContent as utmContent,
  trafficSource.campaign as utmCampaign,
  trafficSource.isTrueDirect as isTrueDirectSession,
  trafficSource.keyword as utmKeyword,
  trafficSource.medium as utmMedium,
  trafficSource.source as utmSource,
  IF((SELECT
     MAX(IF(v2ProductCategory in ("postpaid","smartphone"), 1, 0)) as isCommercialSession
   FROM UNNEST(hits) as h, UNNEST(product) as p) = 1,1,0) as isCommercialSession,
  (SELECT 
     STRING_AGG(page.pagePath) as landingpage
   FROM UNNEST(hits) as h WHERE h.type = "PAGE" AND h.isEntrance = TRUE) as landingpage,
  IF(totals.bounces IS NULL, 0, 1) as isBounce,
  --COUNT(DISTINCT(CONCAT(fullVisitorId, CAST(visitStartTime AS STRING)))) AS sessions,
  --SUM((SELECT COUNT(DISTINCT(transaction.transactionId)) FROM UNNEST(hits) WHERE transaction.transactionId IS NOT NULL)) AS transactions,
  (SELECT 
     MAX(IF(REGEXP_CONTAINS(page.pagePath, r"^/mein-o2/.*|^/service/.*|^/kundencenter/.*") OR 
        page.hostname IN ("freundewerben.o2online.de", "login.o2online.de") OR
        (hcd.index = 193 AND hcd.value in ("ecare","ecare-postpaid", "ecare-prepaid","welcome")), 1, 0)) as isServiceTraffic
   FROM UNNEST(hits), UNNEST(customdimensions) as hcd) as isServiceTraffic,
  (SELECT 
     MAX(IF(page.pagePath LIKE "%/dsl-festnetz/%" OR (hcd.index = 70 AND hcd.value LIKE "'%dsl%"), 1, 0 )) as isDSLSession
   FROM UNNEST(hits), UNNEST(customdimensions) as hcd) as isDSLSession,
  (SELECT 
     SIGN(COUNT(1))
   FROM UNNEST(hits), UNNEST(customdimensions) as hcd WHERE hcd.index = 193 AND hcd.value LIKE "%pixie%" LIMIT 1) as isPixieSession,
  (SELECT 
     CAST(STRING_AGG(DISTINCT hcd.value) = 'pixie' as INT64)
   FROM UNNEST(hits), UNNEST(customdimensions) as hcd WHERE hcd.index = 193) as isPixieOnlySession,
  (SELECT 
     IF(isEntrance = true AND REGEXP_CONTAINS(page.pagePath, r"eshop.*confirmation|e-shop/bestellung|basket|warenkorb|cart|checkout|/check24de|/check24|/preis24|/mediamarkt|/saturn|reservierung"), 1, 0 ) as isDirectBuySession
   FROM UNNEST(hits) LIMIT 1) as isDirectBuySession,
  (SELECT
     MAX(IF(hcd.value = "econtract",1,0)) as isVVLSession
   FROM UNNEST(hits), UNNEST(customdimensions) as hcd WHERE hcd.index = 193) as isVVLSession,
  IF((SELECT MAX(
  IF((sessionCD.index = 175
  AND sessionCD.value LIKE '%Kunde%') OR
  trafficSource.medium LIKE '%CRM%' OR
  trafficSource.source LIKE '%CRM%', 1, 0)) FROM UNNEST(customDimensions) AS sessionCD) = 1,1,0) as isCustomerSession,
  (SELECT
     MAX(IF(REGEXP_CONTAINS(page.pagePath, r"eshop.*basket|e-shop/warenkorb|e-shop/checkout|e-shop/bestellung|e-store/cart"), 1, 0)) as isCartAdd
   FROM UNNEST(hits)) as isCartAdd,
  (SELECT
     MAX(IF(REGEXP_CONTAINS(page.pagePath, r"eshop.*confirmation|/pixie/confirmation/|/e-shop/bestellung/") OR
        eventInfo.eventCategory = "Summary Paynow", 1, 0)) as isEShopTransaction
   FROM UNNEST(hits)) as isEShopTransaction
FROM
  date_range AS s
  LEFT JOIN (
    SELECT clientId, visitId, h.transaction.transactionId gaTransactId,
           p.v2ProductCategory as transactionItemCats,
           (SELECT pCd.value as productCheckoutPromocode FROM UNNEST(p.customDimensions) as pCd WHERE pCd.index = 62) as productCheckoutPromocode
    FROM date_range s, UNNEST(hits) as h, UNNEST(h.product) as p-- , UNNEST(p.customDimensions) as pCd
    WHERE h.transaction.transactionId IS NOT NULL --AND (pCd.index = 62 OR NOT EXISTS (SELECT index FROM p.customDimensions WHERE index = 62))
    GROUP BY clientId, visitId, gaTransactId, transactionItemCats, productCheckoutPromocode
  ) AS tr ON tr.visitId = s.visitId AND tr.clientId = s.clientId
 
GROUP BY
  channelGrouping,
  clientId,
  visitId,
  gaTransactId,
  transactionItemCats,
  productCheckoutPromocode,
  GCLID,
  sessionDeviceCategory,
  landingpage,
  date,
  isCommercialSession,
  isServiceTraffic,
  isDSLSession,
  isDirectBuySession,
  isVVLSession,
  isCustomerSession,
  isPixieSession,
  isPixieOnlySession,
  isCartAdd,
  isEShopTransaction,
  utmContent,
  utmCampaign,
  isTrueDirectSession,
  utmKeyword,
  utmMedium,
  utmSource,
  isBounce
)
Select * from mopGA  where isEShopTransaction  =1

,date,clientId,visitId,GCLID,gaTransactId,transactionItemCats,productCheckoutPromocode,sessionDeviceCategory,channelGrouping,utmContent,utmCampaign,isTrueDirectSession,utmKeyword,utmMedium,utmSource,isCommercialSession,landingpage,isBounce,isServiceTraffic,isDSLSession,isPixieSession,isPixieOnlySession,isDirectBuySession,isVVLSession,isCustomerSession,isCartAdd,isEShopTransaction
0,20200801,663639929.1596214287,1596242196,None,o2-N-MblBHGpnQ0WSIm2ivBJqhQ-1,hw-only_smartphone,NO_PROMOTION,mobile,AllOther_undefined_internal_partnering_eRetail,(not set),kundenservice_hotline,True,(not set),NF_App,wallet,0,/,0,0,0,0,0,0,0,0,1,1
1,20200801,1673161515.1596293890,1596293890,None,o2-N-5bmb-lhnTAGSwy7vZbtFTw-1,hw-only_tablet,NO_PROMOTION,mobile,Direct,(not set),(not set),True,(not set),(none),(direct),0,/mein-o2/,0,1,0,0,0,0,0,0,1,1
2,20200801,1673161515.1596293890,1596293890,None,o2-N-PCCzkLw8RJ-R0RcVXWi5ZA-1,hw-only_tablet,NO_PROMOTION,mobile,Direct,(not set),(not set),True,(not set),(none),(direct),0,/mein-o2/,0,1,0,0,0,0,0,0,1,1
3,20200801,584222439.1593334307,1596290411,None,o2-B-007RN46SSyaLvXk-nAHMBQ-1,hw-only_smartphone,NO_PROMOTION,mobile,Direct,O2_DTI_APP_TEF_11115,top-angebote,None,angebote,app,O2_DTI_APP_TEF_11115,0,/vorteile/fuer-kunden/handy/top-angebote/,0,1,0,0,0,0,0,1,1,1
4,20200801,584222439.1593334307,1596290411,None,o2-B-Cb8da-pwS26LvyMXrvev3g-1,hw-only_smartphone,NO_PROMOTION,mobile,Direct,O2_DTI_APP_TEF_11115,top-angebote,None,angebote,app,O2_DTI_APP_TEF_11115,0,/vorteile/fuer-kunden/handy/top-angebote/,0,1,0,0,0,0,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1484,20200801,1164616795.1596306974,1596306973,EAIaIQobChMI1NO96NL66gIVBap3Ch2KQQd-EAAYASAAEg...,o2-C-bt29-aPnSJyZ7V_dulXHBg-1,postpaid_sim-only,NO_PROMOTION,desktop,SEA Lower Funnel,o2 Handy- & DSL-Tarife,0. Brand exact > o2,None,o2,cpc,google,0,/,0,0,0,0,0,0,0,0,1,1
1485,20200801,1314109013.1596252734,1596252734,None,o2-B-pfBqyNKoQtm03PhYFwPIfg-2,postpaid_bundle,NO_PROMOTION,mobile,Direct,(not set),(not set),True,(not set),(none),(direct),0,/sub-login/auth/login/,0,1,1,0,0,0,0,1,1,1
1486,20200801,1656865399.1595771876,1596291466,EAIaIQobChMI0rGvi5n66gIVwuF3Ch2TuwHQEAAYASAAEg...,o2-C--6yEukpBQ-uytnffMPn4AQ-1,postpaid_sim-only,NO_PROMOTION,mobile,SEA Lower Funnel,o2 Handy- & DSL-Tarife,0. Brand exact > o2,None,o2,cpc,google,0,/,0,1,0,0,0,0,0,0,1,1
1487,20200801,1656865399.1595771876,1596291466,EAIaIQobChMI0rGvi5n66gIVwuF3Ch2TuwHQEAAYASAAEg...,o2-C-nYV5KT97T2ChdzWg0JKZhQ-1,postpaid_sim-only,NO_PROMOTION,mobile,SEA Lower Funnel,o2 Handy- & DSL-Tarife,0. Brand exact > o2,None,o2,cpc,google,0,/,0,1,0,0,0,0,0,0,1,1


**xxx neue spalten**

In [ ]:
%%bigquery --project tefgfs
#https://github.com/Telefonica-Germany/TEFBigQuery/blob/master/mop-gaSource-v6-bl.sql

WITH date_range as 
(
SELECT * FROM  `tefgfs.103498486.ga_sessions_20200801` Limit 200000
#WHERE _TABLE_SUFFIX = FORMAT_DATE("%Y%m%d", DATE_SUB(@run_date, INTERVAL 1 DAY)) 
)


SELECT
  date,
  clientId,
  visitId,


        device.operatingSystem as p2,  
        #trafficSource  as p3,
        device.browser as p9,
        device.deviceCategory as p3,
        device.mobileDeviceModel  as p4,
        device.mobileDeviceBranding  as p5,
         trafficSource.adwordsClickInfo.adNetworkType as p6,
        geoNetwork.city  as p7,
        geoNetwork.region as p8

 FROM date_range, Unnest(hits) as h 
 # where  REGEXP_CONTAINS(pp, r"^/subscriptionId.*") 
group by p2, p3, p4,p5, p6,p7,p8,p9

Executing query with job ID: 182714d7-7d7b-43c9-b3bd-826066d07006
Query executing: 0.56s


ERROR:
 400 Unrecognized name: s at [11:3]

(job ID: 182714d7-7d7b-43c9-b3bd-826066d07006)

                             -----Query Job SQL Follows-----                              

    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |
   1:#https://github.com/Telefonica-Germany/TEFBigQuery/blob/master/mop-gaSource-v6-bl.sql
   2:
   3:WITH date_range as 
   4:(
   5:SELECT * FROM  `tefgfs.103498486.ga_sessions_20200801` Limit 200000
   6:#WHERE _TABLE_SUFFIX = FORMAT_DATE("%Y%m%d", DATE_SUB(@run_date, INTERVAL 1 DAY)) 
   7:)
   8:
   9:
  10:SELECT
  11:  s.date,
  12:  s.clientId,
  13:  s.visitId,
  14:
  15:
  16:        device.operatingSystem as p2,  
  17:        #trafficSource  as p3,
  18:        device.browser as p9,
  19:        device.deviceCategory as p3,
  20:        device.mobileDeviceModel  as p4,
  21:        device.mobileDeviceBranding  as p5,
  22:         trafficSource.adwordsClickInfo.adNetworkType as p6,
  23:        geoNetwork.c

In [ ]:
%%bigquery --project tefgfs
#https://github.com/Telefonica-Germany/TEFBigQuery/blob/master/mop-gaSource-v6-bl.sql

WITH date_range as 
(
SELECT * FROM  `tefgfs.103498486.ga_sessions_20200801`
#WHERE _TABLE_SUFFIX = FORMAT_DATE("%Y%m%d", DATE_SUB(@run_date, INTERVAL 1 DAY)) 
)
#subscriptionId
# ( SELECT 
#     Max(IF(REGEXP_CONTAINS(page.pagePath, r"^/mein-o2/.*|^/service/.*|^/kundencenter/.*") OR 
#        page.hostname IN ("freundewerben.o2online.de", "login.o2online.de") OR
#        (hcd.index = 193 AND hcd.value in ("ecare","ecare-postpaid", "ecare-prepaid","welcome")), 1, 0)) 
#      FROM UNNEST(hits)as h, UNNEST(customdimensions) as hcd) 
#                  as isServiceTraffic

,serviceData as
(
    SELECT clientId, visitId, h.transaction.transactionId gaTransactId,
           p.v2ProductCategory as transactionItemCats,
           p.v2ProductName as pName,
           
 ( SELECT 
     Max(IF(REGEXP_CONTAINS(page.pagePath, r"^/mein-o2/.*|^/service/.*|^/kundencenter/.*") OR 
        page.hostname IN ("freundewerben.o2online.de", "login.o2online.de") OR
        (hcd.index = 193 AND hcd.value in ("ecare","ecare-postpaid", "ecare-prepaid","welcome")), 1, 0)) 
      FROM UNNEST(hits)as h, UNNEST(customdimensions) as hcd) 
                  as isServiceTraffic

    FROM date_range s, UNNEST(hits) as h, UNNEST(h.product) as p #-- , UNNEST(p.customDimensions) as pCd
    WHERE h.transaction.transactionId IS NOT NULL #--AND (pCd.index = 62 OR NOT EXISTS (SELECT index FROM p.customDimensions WHERE index = 62))
    
    GROUP BY clientId, visitId, gaTransactId, transactionItemCats, pName,isServiceTraffic
)
select * from  serviceData   where isServiceTraffic = 1

,clientId,visitId,gaTransactId,transactionItemCats,pName,isServiceTraffic
0,498616888.1596278680,1596278681,bk-econtract-o2online-1596278792.12222658-1,postpaid_sim-only-vvl,o2 Free L (2019),1
1,1837060240.1594742944,1596277638,bk-econtract-o2online-1596277803.60654039-1,postpaid_sim-only-vvl,o2 Free M Online (2018),1
2,1375570731.1574363530,1596264371,bk-econtract-o2online-1596264605.446573445-1,postpaid_sim-only-tw,o2 Free L Boost,1
3,1834392939.1537431298,1596302918,bk-econtract-o2online-1596303479.1432330914-1,postpaid_sim-only-vvl,o2 Free M (2018),1
4,1172092500.1529223812,1596300048,o2-B-wT-eZghQRG2GzEF-YLsziw-1,hw-only_smartphone,hw-only,1
...,...,...,...,...,...,...
1501,1574231159.1596298737,1596301864,o2-B-Z6nF003xTymxEubxtQqwAA-2,hw-only_smartphone,hw-only,1
1502,454650328.1564625480,1596302360,o2-B-1kp-OJSfT-i4yTQSYG1T5A-1,hw-only_smartphone,hw-only,1
1503,150455331.1566929274,1596291201,bk-econtract-o2online-1596292105.1241166403-1,postpaid_sim-only-vvl,o2 Free L Online (2018),1
1504,1388055812.1596268281,1596268281,bk-econtract-o2online-1596269117.830422192-1,postpaid_sim-only-vvl,o2 Free S Boost Plus,1


In [ ]:
%%bigquery --project tefgfs

SELECT page.pagePath as pp,
        page.pageTitle as p1, 
        device.operatingSystem as p2,  
        #trafficSource  as p3,
        device.browser as p9,
        device.deviceCategory as p3,
        device.mobileDeviceModel  as p4,
        device.mobileDeviceBranding  as p5,
         trafficSource.adwordsClickInfo.adNetworkType as p6,
        geoNetwork.city  as p7,
        geoNetwork.region as p8

 FROM `tefgfs.103498486.ga_sessions_20201004`, Unnest(hits) as h 
 # where  REGEXP_CONTAINS(pp, r"^/subscriptionId.*") 
group by pp,p1,p2, p3, p4,p5, p6,p7,p8,p9
     

,pp,p1,p2,p9,p3,p4,p5,p6,p7,p8
0,/e-store/,"o2 | Mobilfunknetz, Handytarife, Top-Smartphon...",Android,Chrome,mobile,H815,LG,None,Mainz,Rhineland-Palatinate
1,/postpaid/econtract/auswahlseite/view/,o2.de,Android,Android Webview,mobile,SM-A600FN,Samsung,None,Hamburg,Hamburg
2,/e-store/hardware/,Handy mit Vertrag: Neue Smartphones günstig be...,Android,Chrome,tablet,B3-A32,Acer,Google Search,Hanover,Lower Saxony
3,/postpaid/econtract/auswahlseite/view/,o2.de,Android,Android Webview,mobile,BLA-L29,Huawei,None,Hanover,Lower Saxony
4,/sub-freischalten/freischalten/success/,Freischalten,Android,Chrome,mobile,SM-A320FL,Samsung,None,Lubeck,Schleswig-Holstein
...,...,...,...,...,...,...,...,...,...,...
518186,/e-shop/samsung/samsung-galaxy-s20-4g-mit-tabl...,Samsung Galaxy S20 4G mit Tablet mit Vertrag |...,Android,Android Webview,mobile,SM-A750FN,Samsung,None,Bernau bei Berlin,Brandenburg
518187,/e-shop/samsung/samsung-galaxy-s20-plus-4g-det...,Samsung Galaxy S20+ mit Vertrag | Günstig kauf...,Android,Chrome,mobile,SM-A750FN,Samsung,Google Search,Nettetal,North Rhine-Westphalia
518188,/e-shop/samsung/samsung-galaxy-watch-active-si...,Samsung Galaxy Watch Active mit Vertrag | Güns...,Android,Samsung Internet,mobile,SM-A750FN,Samsung,Cross-network,(not set),Saxony-Anhalt
518189,/e-shop/samsung/samsung-galaxy-a51-weiss-details/,Samsung Galaxy A51 mit Vertrag | Günstig kaufe...,Android,Android Webview,mobile,SM-A750FN,Samsung,None,Hamburg,Hamburg


**Hol alle CustomDim 150  items heraus (das sind alle url-parameter)**

In [ ]:
%%bigquery --project tefgfs

with k1data as (
SELECT 
     count( CONCAT( fullvisitorid,"-",CAST(visitStartTime AS string))) as visits,
   #  if (h.transaction.transactionId is null, "",h.transaction.transactionId) as gaTransactId,
     count(h.transaction.transactionId) as transactions, 
      #h.transaction.transactionId IS NOT NULL --
   
    (SELECT 
     MAX(IF(REGEXP_CONTAINS(page.pagePath, r"^/mein-o2/.*|^/service/.*|^/kundencenter/.*|^/priority/.*") OR 
        page.hostname IN ("freundewerben.o2online.de", "login.o2online.de", "edel.o2online.de") OR
        (hcd.index = 193 AND hcd.value in ("ecare","ecare-postpaid","ecare-prepaid","welcome","blog","hilfe","freischalten","b2b","econtract","mehr-o2","root-priority")), 1, 0)) as isServiceTraffic
   FROM UNNEST(hits) as h, UNNEST(h.customdimensions) as hcd) as isServiceTraffic,
   
   
    (SELECT   STRING_AGG(DISTINCT hcd.value)
      FROM UNNEST(hits) as h, UNNEST(customdimensions) as hcd    WHERE hcd.index = 150)          as custDim150, 
    
       page.pagePath as pp,
        page.pageTitle as p1, 
        device.operatingSystem as p2,  
        #trafficSource  as p3,
        device.browser as p9,
        device.deviceCategory as p3,
        device.mobileDeviceModel  as p4,
        device.mobileDeviceBranding  as p5,
         trafficSource.adwordsClickInfo.adNetworkType as p6,
        geoNetwork.city  as p7,
        geoNetwork.region as p8
        

 FROM (select * from `tefgfs.103498486.ga_sessions_20201004` ), Unnest(hits) as h #,UNNEST(h.product) as p , UNNEST(p.customDimensions) as pCd
 # where  REGEXP_CONTAINS(pp, r"^/subscriptionId.*") 
group by pp,p1,p2, p3, p4,p5, p6,p7,p8,p9,custDim150,isServiceTraffic
#order by 2 desc 
order by custDim150 
)
#select sum(visits) as visits, sum (transactions) as transactions from k1data
#standardSQL
# Extract query parameters from a URL as ARRAY in BigQuery; standard-sql; 2018-04-08
# @see http://www.pascallandau.com/bigquery-snippets/extract-url-parameters-array/
, subData as (
SELECT 
   custDim150 ,
   REGEXP_EXTRACT_ALL(custDim150,r'(?:\?|&)((?:[^=]+)=(?:[^&]*))') as params,
  REGEXP_EXTRACT_ALL(custDim150,r'(?:\?|&)(?:([^=]+)=(?:[^&]*))') as keys,
  REGEXP_EXTRACT_ALL(custDim150,r'(?:\?|&)(?:(?:[^=]+)=([^&]*))') as values
  #description
FROM k1data #  Where keys1 = "goto"
)
Select distinct(k) from subData, Unnest(keys) as k #where k = "goto" 
order by 1 asc



,k
0,_mmcb
1,_mmcbid
2,acr_values
3,adgroupname
4,appl
...,...
74,utm_term
75,vendor
76,vertragsart
77,wfid


suche in der customDim 150 nach nem key-value - paar (urlParameter) mit subscriptionId als parameter

In [ ]:
WITH
  k1data AS (
  SELECT
    COUNT( CONCAT( fullvisitorid,"-",CAST(visitStartTime AS string))) AS visits,
    #  if (h.transaction.transactionId is null, "",h.transaction.transactionId) as gaTransactId,
    COUNT(h.TRANSACTION.transactionId) AS transactions,
    #h.transaction.transactionId IS NOT NULL --
    (
    SELECT
      MAX(
      IF
        (REGEXP_CONTAINS(page.pagePath, r"^/mein-o2/.*|^/service/.*|^/kundencenter/.*|^/priority/.*")
          OR page.hostname IN ("freundewerben.o2online.de",
            "login.o2online.de",
            "edel.o2online.de")
          OR (hcd.index = 193
            AND hcd.value IN ("ecare",
              "ecare-postpaid",
              "ecare-prepaid",
              "welcome",
              "blog",
              "hilfe",
              "freischalten",
              "b2b",
              "econtract",
              "mehr-o2",
              "root-priority")),
          1,
          0))
    FROM
      UNNEST(hits) AS h,
      UNNEST(h.customdimensions) AS hcd) AS isServiceTraffic,
    (
    SELECT
      STRING_AGG(DISTINCT hcd.value)
    FROM
      UNNEST(hits) AS h,
      UNNEST(customdimensions) AS hcd
    WHERE
      hcd.index = 150) AS custDim150,
    page.pagePath AS pp,
    page.pageTitle AS p1,
    device.operatingSystem AS p2,
    #trafficSource  as p3,
    device.browser AS p9,
    device.deviceCategory AS p3,
    device.mobileDeviceModel AS p4,
    device.mobileDeviceBranding AS p5,
    trafficSource.adwordsClickInfo.adNetworkType AS p6,
    geoNetwork.city AS p7,
    geoNetwork.region AS p8
  FROM (
    SELECT
      *
    FROM
      `tefgfs.103498486.ga_sessions_20201004` ),
    UNNEST(hits) AS h #,UNNEST(h.product) as p , UNNEST(p.customDimensions) as pCd
    # where  REGEXP_CONTAINS(pp, r"^/subscriptionId.*")
  GROUP BY
    pp,
    p1,
    p2,
    p3,
    p4,
    p5,
    p6,
    p7,
    p8,
    p9,
    custDim150,
    isServiceTraffic
    #order by 2 desc
  ORDER BY
    custDim150 )
  #select sum(visits) as visits, sum (transactions) as transactions from k1data
  #standardSQL
  # Extract query parameters from a URL as ARRAY in BigQuery; standard-sql; 2018-04-08
  # @see http://www.pascallandau.com/bigquery-snippets/extract-url-parameters-array/
  ,
  subData AS (
  SELECT
    isServiceTraffic,
    custDim150,
    REGEXP_EXTRACT_ALL(custDim150,r'(?:\?|&)((?:[^=]+)=(?:[^&]*))') AS params,
    REGEXP_EXTRACT_ALL(custDim150,r'(?:\?|&)(?:([^=]+)=(?:[^&]*))') AS keys,
    REGEXP_EXTRACT_ALL(custDim150,r'(?:\?|&)(?:(?:[^=]+)=([^&]*))') AS
  VALUES
    #description
  FROM
    k1data #  Where keys1 = "goto"
    )
SELECT
  *
FROM
  subData,
  UNNEST(keys) AS k
WHERE
  k LIKE "%subscription%" #and isServiceTraffic = 1
ORDER BY
  1 ASC